# Estimate sentence probability with BERT
## Calculating probability more properly:
P_f, P_b: Probability forward pass, backward pass, respectively
```
P_f = P(w_0) * P(w_1|w_0) * P(w_2|w_0, w_1) * ... * P(w_N)
P_b = P(w_N-1|w_N) * P(w_N-2|w_N-1, w_N) * ... * P(w_0|w_1, w_2, ... ,w_N)
```
In this notebook, probabilities are not normalized by sentence length, so `P_f`, `P_b` become smaller as the sentence length increases.

Finally, the sentence probability P(S) is the geometric mean of forward and backwards probabilities:
```
P(S) = (mean(P_f(S)) * mean(P_b(S))) ^ (1/2)
```

In [1]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForMaskedLM

BOS_TOKEN = '[CLS]'
EOS_TOKEN = '[SEP]'
MASK_TOKEN = '[MASK]'

# Load pre-trained model (weights)
with torch.no_grad():
    model = BertForMaskedLM.from_pretrained('bert-large-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
def print_top_predictions(probs, k=5):
    probs = probs.detach().numpy()
    top_indexes = np.argpartition(probs, -k)[-k:]
    sorted_indexes = top_indexes[np.argsort(-probs[top_indexes])]
    top_tokens = tokenizer.convert_ids_to_tokens(sorted_indexes)
    print(f"Ordered top predicted tokens: {top_tokens}")
    print(f"Ordered top predicted values: {probs[sorted_indexes]}")

In [3]:
def get_sentence_prob(sentence, verbose=True):
    # Pre-process sentence, adding special tokens
    tokenized_input = tokenizer.tokenize(sentence)
    if tokenized_input[0] != BOS_TOKEN:
        tokenized_input.insert(0, BOS_TOKEN)
    if tokenized_input[-1] != EOS_TOKEN:
        tokenized_input.append(EOS_TOKEN)
    sent_len = len(tokenized_input)
    ids_input = tokenizer.convert_tokens_to_ids(tokenized_input)
    print(f"Processing sentence: {tokenized_input}\n")
    
    sm = torch.nn.Softmax(dim=0) # used to convert last hidden state to probs
    
    log_sent_prob_forward = 0
    log_sent_prob_backwards = 0
    # Mask non-special tokens in forward and backwards directions; calculate their probabilities
    for i in range(1, len(tokenized_input) - 1):  # Don't loop first and last tokens
        probs_forward = get_directional_prob(sm, tokenized_input, i, 'forward', verbose=verbose)
        probs_backwards = get_directional_prob(sm, tokenized_input, i, 'backwards', verbose=verbose)
        log_prob_forward = probs_forward[ids_input[i]]  # Prediction for masked word
        log_prob_forward = np.log10(log_prob_forward.detach().numpy())
        log_prob_backwards = probs_backwards[ids_input[i]]  # Prediction for masked word
        log_prob_backwards = np.log10(log_prob_backwards.detach().numpy())
        log_sent_prob_forward += log_prob_forward
        log_sent_prob_backwards += log_prob_backwards

        if verbose:
            print(f"Word: {tokenized_input[i]} \t Log-Prob_forward: {log_prob_forward}; Log-Prob_backwards: {log_prob_backwards}")

    # Obtain geometric average of forward and backward probs
    log_geom_mean_sent_prob = 0.5 * (log_sent_prob_forward + log_sent_prob_backwards)
    if verbose:
        print(f"Raw forward sentence probability: {log_sent_prob_forward}")
        print(f"Raw backward sentence probability: {log_sent_prob_backwards}\n")
        print(f"Average normalized sentence prob: {log_geom_mean_sent_prob}\n")
    return np.power(10, log_geom_mean_sent_prob)

In [4]:
def get_directional_prob(sm, tokenized_input, i, direction, verbose=False):
    current_tokens = tokenized_input[:]
    if direction == 'backwards':
        current_tokens[1:i+1] = [MASK_TOKEN for j in range(i)]
    elif direction == 'forward':
        current_tokens[i:-1] = [MASK_TOKEN for j in range(len(tokenized_input) - 1 - i)]
    else:
        print("Direction can only be 'forward' or 'backwards'")
        exit()
    if verbose: 
        print()
        print(current_tokens)
        
    masked_input = torch.tensor([tokenizer.convert_tokens_to_ids(current_tokens)])
    predictions = model(masked_input)
    predictions = predictions[0]
    probs = sm(predictions[0, i]) # Softmax to get probabilities
    if verbose: 
        print_top_predictions(probs)
    
    return probs # Model predictions

In [18]:
get_sentence_prob("He was receiving quite a ridiculous the.")

Processing sentence: ['[CLS]', 'he', 'was', 'receiving', 'quite', 'a', 'ridiculous', 'the', '.', '[SEP]']


['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['"', "'", '-', 'oh', 'then']
Ordered top predicted values: [0.32555664 0.064494   0.0138153  0.01192483 0.01160424]

['[CLS]', '[MASK]', 'was', 'receiving', 'quite', 'a', 'ridiculous', 'the', '.', '[SEP]']
Ordered top predicted tokens: ['i', 'she', 'he', 'it', 'jack']
Ordered top predicted values: [0.3033052  0.28302258 0.26991323 0.00534671 0.00106846]
Word: he 	 Log-Prob_forward: -2.9956727027893066; Log-Prob_backwards: -0.5687758922576904

['[CLS]', 'he', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['was', 'looked', 'started', 'said', 'stopped']
Ordered top predicted values: [0.0795251  0.04593899 0.0360042  0.03072959 0.02925682]

['[CLS]', '[MASK]', '[MASK]', 'receiving', 'quite', 'a

3.541401918411717e-22

In [5]:
get_sentence_prob("penguins are birds.")
get_sentence_prob("penguins have wings.")
get_sentence_prob("wings are useful.")

Processing sentence: ['[CLS]', 'penguins', 'are', 'birds', '.', '[SEP]']


['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['the', '"', 'a', 'i', 'no']
Ordered top predicted values: [0.09850086 0.06786957 0.05361035 0.03242255 0.01996884]

['[CLS]', '[MASK]', 'are', 'birds', '.', '[SEP]']
Ordered top predicted tokens: ['there', 'they', 'some', 'these', 'all']
Ordered top predicted values: [0.50550175 0.17322007 0.05124235 0.04546349 0.01676534]
Word: penguins 	 Log-Prob_forward: -5.483602046966553; Log-Prob_backwards: -4.805809020996094

['[CLS]', 'penguins', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['.', '&', 'illustrated', 'and', 'world']
Ordered top predicted values: [0.55944806 0.0238416  0.02219316 0.01686097 0.01659434]

['[CLS]', '[MASK]', '[MASK]', 'birds', '.', '[SEP]']
Ordered top predicted tokens: ['the', 'of', 'for', '.', 'wild']
Ordered top predicted values: [0.05914232 0.02854817 0.02677561 0.02595682 0.

4.4051026738671185e-13

In [29]:
get_sentence_prob("The kids eat the candy.")
get_sentence_prob("The kids eat the apple.")
get_sentence_prob("The kids ate the apple.")
get_sentence_prob("The kids ate the apple quickly.")
get_sentence_prob("The kids ate the apple slowly.")

Processing sentence: ['[CLS]', 'the', 'kids', 'eat', 'the', 'candy', '.', '[SEP]']


['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['and', '-', '.', '"', '(']
Ordered top predicted values: [0.02624164 0.02091097 0.01713027 0.01695242 0.01515674]

['[CLS]', '[MASK]', 'kids', 'eat', 'the', 'candy', '.', '[SEP]']
Ordered top predicted tokens: ['the', 'other', 'some', 'my', 'little']
Ordered top predicted values: [0.96681714 0.00511999 0.00450562 0.00371905 0.00316838]
Word: the 	 Log-Prob_forward: -2.188230037689209; Log-Prob_backwards: -0.014655658043920994

['[CLS]', 'the', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['weather', 'film', 'score', 'book', 'story']
Ordered top predicted values: [0.04039879 0.02398504 0.01847865 0.0163132  0.01564541]

['[CLS]', '[MASK]', '[MASK]', 'eat', 'the', 'candy', '.', '[SEP]']
Ordered top predicted tokens: ['should', 'will', 'can', 'would', 'j

Ordered top predicted tokens: ['of', 'at', 'in', 'and', 'on']
Ordered top predicted values: [0.23147754 0.08573806 0.08245695 0.06939063 0.05840191]
Word: ate 	 Log-Prob_forward: -2.647407054901123; Log-Prob_backwards: -1.8155173063278198

['[CLS]', 'the', 'kids', 'ate', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['the', 'their', 'in', 'a', 'at']
Ordered top predicted values: [0.1605707  0.15663709 0.02839522 0.02668495 0.01951117]

['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'apple', '.', '[SEP]']
Ordered top predicted tokens: ['the', 'big', 'golden', 'red', 'black']
Ordered top predicted values: [0.08482028 0.06450091 0.05034276 0.04952531 0.04622569]
Word: the 	 Log-Prob_forward: -0.7943336963653564; Log-Prob_backwards: -1.071500301361084

['[CLS]', 'the', 'kids', 'ate', 'the', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['food', 'cookies', 'bread', 'cake', 'popcorn']
Ordered top predicted values: [0.18070029 0.07747439 0.05205146 0.046

Ordered top predicted tokens: ['the', 'his', 'an', 'her', 'my']
Ordered top predicted values: [0.6511448  0.1720026  0.10000458 0.04762108 0.01570826]
Word: the 	 Log-Prob_forward: -0.8795525431632996; Log-Prob_backwards: -0.1863224357366562

['[CLS]', 'the', 'kids', 'ate', 'the', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['whole', 'same', 'meat', 'sweet', 'first']
Ordered top predicted values: [0.11392502 0.04679374 0.02004475 0.01987989 0.01665526]

['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'slowly', '.', '[SEP]']
Ordered top predicted tokens: [',', 'very', 'down', 'away', 'out']
Ordered top predicted values: [0.4479095  0.24251206 0.06397119 0.03145403 0.02032288]
Word: apple 	 Log-Prob_forward: -2.2021851539611816; Log-Prob_backwards: -5.784820079803467

['[CLS]', 'the', 'kids', 'ate', 'the', 'apple', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['pie', 'together', 'first', 'cake', 'too']
Ordered top predicted values: [0.14

6.025887973582306e-16

In [27]:
get_sentence_prob("The kids eat the candy.")
get_sentence_prob("kids eat.")
get_sentence_prob("eat kids.")
get_sentence_prob("kids eat candy.")
get_sentence_prob("the kids eat.")
get_sentence_prob("small kids eat.")

Processing sentence: ['[CLS]', 'the', 'kids', 'eat', 'the', 'candy', '.', '[SEP]']


['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['and', '-', '.', '"', '(']
Ordered top predicted values: [0.02624164 0.02091097 0.01713027 0.01695242 0.01515674]

['[CLS]', '[MASK]', 'kids', 'eat', 'the', 'candy', '.', '[SEP]']
Ordered top predicted tokens: ['the', 'other', 'some', 'my', 'little']
Ordered top predicted values: [0.96681714 0.00511999 0.00450562 0.00371905 0.00316838]
Word: the 	 Log-Prob_forward: -2.188230037689209; Log-Prob_backwards: -0.014655658043920994

['[CLS]', 'the', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['weather', 'film', 'score', 'book', 'story']
Ordered top predicted values: [0.04039879 0.02398504 0.01847865 0.0163132  0.01564541]

['[CLS]', '[MASK]', '[MASK]', 'eat', 'the', 'candy', '.', '[SEP]']
Ordered top predicted tokens: ['should', 'will', 'can', 'would', 'j

Ordered top predicted tokens: ['vol', 'press', 'p', 'j', 'ii']
Ordered top predicted values: [0.06490542 0.05437048 0.03844456 0.0262519  0.01432491]
Word: candy 	 Log-Prob_forward: -1.5477046966552734; Log-Prob_backwards: -5.272588729858398

['[CLS]', 'kids', 'eat', 'candy', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['.', ';', '!', '?', '|']
Ordered top predicted values: [9.9884027e-01 1.1573723e-03 2.3371590e-06 4.6723546e-08 2.5806515e-10]

['[CLS]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Ordered top predicted tokens: ['.', ';', '|', '?', '!']
Ordered top predicted values: [9.5748538e-01 4.0964618e-02 1.0687120e-03 2.2205077e-04 2.2124879e-04]
Word: . 	 Log-Prob_forward: -0.0005039555835537612; Log-Prob_backwards: -0.018867850303649902
Raw forward sentence probability: -8.997384486894589
Raw backward sentence probability: -8.530954122543335

Average normalized sentence prob: -8.764169304718962

Processing sentence: ['[CLS]', 'the', 'kids', 'eat', '.', '[SEP]']


['[

1.5414701923956811e-13

In [5]:
get_sentence_prob("Smurfs eat the ancient nuns ungracefully.")
get_sentence_prob("Smurfs eat ancient the nuns ungracefully.")
get_sentence_prob("eat smurfs the ancient nuns ungracefully.")
get_sentence_prob("Smurfs eat the ancient ungracefully nuns.")
get_sentence_prob("Smurfs the eat ancient nuns ungracefully.")

Processing sentence: ['[CLS]', 'sm', '##ur', '##fs', 'eat', 'the', 'ancient', 'nuns', 'un', '##gra', '##ce', '##fully', '.', '[SEP]']

Word: sm 	 Prob_forward: -8.421839714050293; Prob_backwards: -0.08659450709819794
Word: ##ur 	 Prob_forward: -3.792858839035034; Prob_backwards: -10.409788131713867
Word: ##fs 	 Prob_forward: -2.545891523361206; Prob_backwards: -16.56703758239746
Word: eat 	 Prob_forward: -8.448018074035645; Prob_backwards: -12.25107479095459
Word: the 	 Prob_forward: -1.7545280456542969; Prob_backwards: -0.23912009596824646
Word: ancient 	 Prob_forward: -10.828279495239258; Prob_backwards: -9.582014083862305
Word: nuns 	 Prob_forward: -12.031983375549316; Prob_backwards: -17.856948852539062
Word: un 	 Prob_forward: -14.874580383300781; Prob_backwards: -0.000980261480435729
Word: ##gra 	 Prob_forward: -8.647161483764648; Prob_backwards: -3.939512252807617
Word: ##ce 	 Prob_forward: -4.396315097808838; Prob_backwards: -12.9059476852417
Word: ##fully 	 Prob_forward: -0.04

-91.63832396443468

In [46]:
get_sentence_prob("kids eat the red grapes quickly.")
get_sentence_prob("quickly eat the red grapes kids.")
get_sentence_prob("the kids eat the red grapes quickly.")
get_sentence_prob("girls eat the red grapes quickly.")
get_sentence_prob("the red grapes eat kids quickly.")
get_sentence_prob("kids eat red the grapes quickly.")
get_sentence_prob("eat kids the red grapes quickly.")
get_sentence_prob("kids eat the red quickly grapes.")
get_sentence_prob("kids the eat red grapes quickly.")

Processing sentence: ['[CLS]', 'kids', 'eat', 'the', 'red', 'grapes', 'quickly', '.', '[SEP]']

Word: kids 	 Prob_forward: -9.770715713500977; Prob_backwards: -12.24870777130127
Word: eat 	 Prob_forward: -8.85970401763916; Prob_backwards: -4.724191188812256
Word: the 	 Prob_forward: -2.024585723876953; Prob_backwards: -0.48318010568618774
Word: red 	 Prob_forward: -5.991503715515137; Prob_backwards: -5.627716541290283
Word: grapes 	 Prob_forward: -9.902673721313477; Prob_backwards: -14.482468605041504
Word: quickly 	 Prob_forward: -6.720923900604248; Prob_backwards: -12.625097274780273
Word: . 	 Prob_forward: -0.01455748826265335; Prob_backwards: -0.3660288155078888
Raw forward sentence probability: -43.284664280712605
Raw backward sentence probability: -50.55739030241966

Average sentence log-prob: -46.92102729156613

Processing sentence: ['[CLS]', 'quickly', 'eat', 'the', 'red', 'grapes', 'kids', '.', '[SEP]']

Word: quickly 	 Prob_forward: -11.486865043640137; Prob_backwards: -11.55

-58.18911224976182

In [47]:
get_sentence_prob("Colorless green ideas sleep furiously.")
get_sentence_prob("Confused dumb benches eat endlessly.")
get_sentence_prob("Hairless ugly men complain constantly.")

Processing sentence: ['[CLS]', 'color', '##less', 'green', 'ideas', 'sleep', 'furiously', '.', '[SEP]']

Word: color 	 Prob_forward: -10.544276237487793; Prob_backwards: -2.5439460277557373
Word: ##less 	 Prob_forward: -7.516035079956055; Prob_backwards: -8.04877758026123
Word: green 	 Prob_forward: -6.6411590576171875; Prob_backwards: -8.82697582244873
Word: ideas 	 Prob_forward: -12.467616081237793; Prob_backwards: -13.871207237243652
Word: sleep 	 Prob_forward: -9.88399887084961; Prob_backwards: -12.250245094299316
Word: furiously 	 Prob_forward: -9.00595474243164; Prob_backwards: -12.923095703125
Word: . 	 Prob_forward: -0.019949400797486305; Prob_backwards: -0.3660288155078888
Raw forward sentence probability: -56.078989470377564
Raw backward sentence probability: -58.830276280641556

Average sentence log-prob: -57.45463287550956

Processing sentence: ['[CLS]', 'confused', 'dumb', 'benches', 'eat', 'endless', '##ly', '.', '[SEP]']

Word: confused 	 Prob_forward: -12.81374359130859

-52.87689888011664

In [48]:
get_sentence_prob("The test was a success.")
get_sentence_prob("The was test a success.")
get_sentence_prob("The test was success a.")
get_sentence_prob("The party was a success.")
get_sentence_prob("The farewell party was definitely not a success.")

Processing sentence: ['[CLS]', 'the', 'test', 'was', 'a', 'success', '.', '[SEP]']

Word: the 	 Prob_forward: -5.038585662841797; Prob_backwards: -0.0418301522731781
Word: test 	 Prob_forward: -9.275725364685059; Prob_backwards: -5.530876636505127
Word: was 	 Prob_forward: -1.194662094116211; Prob_backwards: -0.10143725574016571
Word: a 	 Prob_forward: -5.843591213226318; Prob_backwards: -1.1096673011779785
Word: success 	 Prob_forward: -0.662120521068573; Prob_backwards: -10.63667106628418
Word: . 	 Prob_forward: -0.05630144849419594; Prob_backwards: -0.3288889527320862
Raw forward sentence probability: -22.070986304432154
Raw backward sentence probability: -17.749371364712715

Average sentence log-prob: -19.910178834572434

Processing sentence: ['[CLS]', 'the', 'was', 'test', 'a', 'success', '.', '[SEP]']

Word: the 	 Prob_forward: -5.038585662841797; Prob_backwards: -8.694866180419922
Word: was 	 Prob_forward: -11.56971263885498; Prob_backwards: -0.7991001009941101
Word: test 	 Prob

-43.25184400938451

In [49]:
get_sentence_prob("He answered unequivocally.")
get_sentence_prob("He answered quickly.", verbose=True)

Processing sentence: ['[CLS]', 'he', 'answered', 'une', '##qui', '##vo', '##cal', '##ly', '.', '[SEP]']

Word: he 	 Prob_forward: -7.592045307159424; Prob_backwards: -1.267844796180725
Word: answered 	 Prob_forward: -8.141104698181152; Prob_backwards: -4.511857986450195
Word: une 	 Prob_forward: -15.309880256652832; Prob_backwards: -0.0012949742376804352
Word: ##qui 	 Prob_forward: -7.458912372589111; Prob_backwards: -7.119812965393066
Word: ##vo 	 Prob_forward: -2.9886157512664795; Prob_backwards: -12.645088195800781
Word: ##cal 	 Prob_forward: -7.105126132955775e-05; Prob_backwards: -10.53508186340332
Word: ##ly 	 Prob_forward: -0.001719282940030098; Prob_backwards: -6.816740036010742
Word: . 	 Prob_forward: -0.0001833010755944997; Prob_backwards: -0.3001890182495117
Raw forward sentence probability: -41.49253202112595
Raw backward sentence probability: -43.19790983572602

Average sentence log-prob: -42.34522092842599

Processing sentence: ['[CLS]', 'he', 'answered', 'quickly', '.', 

-17.174810956348665

In [50]:
get_sentence_prob("The guy with small hands demanded a quid pro quo.")
get_sentence_prob("The guy with small hands demanded an exchange.")

Processing sentence: ['[CLS]', 'the', 'guy', 'with', 'small', 'hands', 'demanded', 'a', 'qui', '##d', 'pro', 'quo', '.', '[SEP]']

Word: the 	 Prob_forward: -3.906177520751953; Prob_backwards: -0.3941221237182617
Word: guy 	 Prob_forward: -9.926000595092773; Prob_backwards: -7.368597984313965
Word: with 	 Prob_forward: -7.264544486999512; Prob_backwards: -2.7467923164367676
Word: small 	 Prob_forward: -9.92628002166748; Prob_backwards: -4.824505805969238
Word: hands 	 Prob_forward: -6.550938129425049; Prob_backwards: -9.48993968963623
Word: demanded 	 Prob_forward: -12.739509582519531; Prob_backwards: -15.551085472106934
Word: a 	 Prob_forward: -7.67767858505249; Prob_backwards: -2.757371664047241
Word: qui 	 Prob_forward: -12.464286804199219; Prob_backwards: -0.6459168195724487
Word: ##d 	 Prob_forward: -1.456640601158142; Prob_backwards: -14.112207412719727
Word: pro 	 Prob_forward: -10.183714866638184; Prob_backwards: -5.321247577667236
Word: quo 	 Prob_forward: -0.00105394807178527

-52.25352312158793

In [51]:
get_sentence_prob("This is a sentence.")
get_sentence_prob("This is a macrame.", verbose=False)
get_sentence_prob("This is a joke.", verbose=False)
get_sentence_prob("Are you kidding me?", verbose=False)


Processing sentence: ['[CLS]', 'this', 'is', 'a', 'sentence', '.', '[SEP]']

Word: this 	 Prob_forward: -8.96243667602539; Prob_backwards: -2.806614875793457
Word: is 	 Prob_forward: -0.9202368259429932; Prob_backwards: -2.717599630355835
Word: a 	 Prob_forward: -2.6885783672332764; Prob_backwards: -1.4664157629013062
Word: sentence 	 Prob_forward: -8.932842254638672; Prob_backwards: -13.830182075500488
Word: . 	 Prob_forward: -0.034496162086725235; Prob_backwards: -0.2193736732006073
Raw forward sentence probability: -21.538590285927057
Raw backward sentence probability: -21.040186017751694

Average sentence log-prob: -21.289388151839375

Processing sentence: ['[CLS]', 'this', 'is', 'a', 'mac', '##ram', '##e', '.', '[SEP]']

Word: this 	 Prob_forward: -8.641555786132812; Prob_backwards: -2.8921501636505127
Word: is 	 Prob_forward: -2.040065050125122; Prob_backwards: -2.0970122814178467
Word: a 	 Prob_forward: -0.29191794991493225; Prob_backwards: -3.2476935386657715
Word: mac 	 Prob_f

-17.183611100091184

In [52]:
get_sentence_prob("Rachel was wearing a lovely satin dress last night.")

Processing sentence: ['[CLS]', 'rachel', 'was', 'wearing', 'a', 'lovely', 'satin', 'dress', 'last', 'night', '.', '[SEP]']

Word: rachel 	 Prob_forward: -11.216014862060547; Prob_backwards: -7.034425258636475
Word: was 	 Prob_forward: -1.3298773765563965; Prob_backwards: -0.0013975725742056966
Word: wearing 	 Prob_forward: -10.184678077697754; Prob_backwards: -2.1435110569000244
Word: a 	 Prob_forward: -0.09067000448703766; Prob_backwards: -0.10912982374429703
Word: lovely 	 Prob_forward: -7.708806037902832; Prob_backwards: -9.332859992980957
Word: satin 	 Prob_forward: -6.861915588378906; Prob_backwards: -10.842044830322266
Word: dress 	 Prob_forward: -3.8740289211273193; Prob_backwards: -13.085378646850586
Word: last 	 Prob_forward: -5.801358699798584; Prob_backwards: -2.8171958923339844
Word: night 	 Prob_forward: -0.052345097064971924; Prob_backwards: -8.598824501037598
Word: . 	 Prob_forward: -0.010135823860764503; Prob_backwards: -0.20058058202266693
Raw forward sentence probabil

-50.647589323169086

In [53]:
get_sentence_prob("Rachel was wearing a lovely satin dress last night.")
get_sentence_prob("Grandma was wearing a lovely satin dress last night.")
get_sentence_prob("Mother was wearing a lovely satin dress last night.")
get_sentence_prob("She was wearing a lovely satin dress last night.")
get_sentence_prob("He was wearing a lovely satin dress last night.")
get_sentence_prob("I was wearing a lovely satin dress last night.")
get_sentence_prob("Angela was wearing a lovely satin dress last night.")
get_sentence_prob("Roberta was wearing a lovely satin dress last night.")
get_sentence_prob("Running was wearing a lovely satin dress last night.")

Processing sentence: ['[CLS]', 'rachel', 'was', 'wearing', 'a', 'lovely', 'satin', 'dress', 'last', 'night', '.', '[SEP]']

Word: rachel 	 Prob_forward: -11.216014862060547; Prob_backwards: -7.034425258636475
Word: was 	 Prob_forward: -1.3298773765563965; Prob_backwards: -0.0013975725742056966
Word: wearing 	 Prob_forward: -10.184678077697754; Prob_backwards: -2.1435110569000244
Word: a 	 Prob_forward: -0.09067000448703766; Prob_backwards: -0.10912982374429703
Word: lovely 	 Prob_forward: -7.708806037902832; Prob_backwards: -9.332859992980957
Word: satin 	 Prob_forward: -6.861915588378906; Prob_backwards: -10.842044830322266
Word: dress 	 Prob_forward: -3.8740289211273193; Prob_backwards: -13.085378646850586
Word: last 	 Prob_forward: -5.801358699798584; Prob_backwards: -2.8171958923339844
Word: night 	 Prob_forward: -0.052345097064971924; Prob_backwards: -8.598824501037598
Word: . 	 Prob_forward: -0.010135823860764503; Prob_backwards: -0.20058058202266693
Raw forward sentence probabil

Word: a 	 Prob_forward: -0.13754932582378387; Prob_backwards: -0.10912982374429703
Word: lovely 	 Prob_forward: -7.054993152618408; Prob_backwards: -9.332859992980957
Word: satin 	 Prob_forward: -6.726216793060303; Prob_backwards: -10.842044830322266
Word: dress 	 Prob_forward: -3.9633948802948; Prob_backwards: -13.085378646850586
Word: last 	 Prob_forward: -5.882959365844727; Prob_backwards: -2.8171958923339844
Word: night 	 Prob_forward: -0.05768701061606407; Prob_backwards: -8.598824501037598
Word: . 	 Prob_forward: -0.013864764012396336; Prob_backwards: -0.20058058202266693
Raw forward sentence probability: -52.242559094913304
Raw backward sentence probability: -58.410240754601546

Average sentence log-prob: -55.326399924757425

Processing sentence: ['[CLS]', 'running', 'was', 'wearing', 'a', 'lovely', 'satin', 'dress', 'last', 'night', '.', '[SEP]']

Word: running 	 Prob_forward: -11.862516403198242; Prob_backwards: -14.420729637145996
Word: was 	 Prob_forward: -7.9794464111328125

-59.89300947828451

In [55]:
get_sentence_prob("The man ate the steak.")
get_sentence_prob("The stake ate the man.")
get_sentence_prob("The man who arrived late ate the steak with a glass of wine.")
get_sentence_prob("The steak was eaten by the man.")
get_sentence_prob("The man was eaten by the stake.")

Processing sentence: ['[CLS]', 'the', 'man', 'ate', 'the', 'steak', '.', '[SEP]']

Word: the 	 Prob_forward: -5.038585662841797; Prob_backwards: -0.05858740955591202
Word: man 	 Prob_forward: -7.065206050872803; Prob_backwards: -9.936883926391602
Word: ate 	 Prob_forward: -8.079832077026367; Prob_backwards: -4.224849700927734
Word: the 	 Prob_forward: -1.8303484916687012; Prob_backwards: -3.6972944736480713
Word: steak 	 Prob_forward: -5.370851039886475; Prob_backwards: -11.686675071716309
Word: . 	 Prob_forward: -0.005581368692219257; Prob_backwards: -0.3288889527320862
Raw forward sentence probability: -27.390404690988362
Raw backward sentence probability: -29.933179534971714

Average sentence log-prob: -28.661792112980038

Processing sentence: ['[CLS]', 'the', 'stake', 'ate', 'the', 'man', '.', '[SEP]']

Word: the 	 Prob_forward: -5.038585662841797; Prob_backwards: -0.1620464026927948
Word: stake 	 Prob_forward: -9.997936248779297; Prob_backwards: -9.75770378112793
Word: ate 	 Prob_

-38.520408786833286

In [56]:
get_sentence_prob("He was born in Berlin.")
get_sentence_prob("He was born in Santiago.")
get_sentence_prob("He was born in France.")
get_sentence_prob("He was born in window.")
get_sentence_prob("He was born in was.")


Processing sentence: ['[CLS]', 'he', 'was', 'born', 'in', 'berlin', '.', '[SEP]']

Word: he 	 Prob_forward: -7.282431602478027; Prob_backwards: -0.2271691858768463
Word: was 	 Prob_forward: -0.49251610040664673; Prob_backwards: -7.152582838898525e-06
Word: born 	 Prob_forward: -9.202768325805664; Prob_backwards: -7.146602153778076
Word: in 	 Prob_forward: -0.004424570593982935; Prob_backwards: -2.567709445953369
Word: berlin 	 Prob_forward: -3.735508441925049; Prob_backwards: -9.517555236816406
Word: . 	 Prob_forward: -6.52096132398583e-05; Prob_backwards: -0.3288889527320862
Raw forward sentence probability: -20.71771425082261
Raw backward sentence probability: -19.787932127739623

Average sentence log-prob: -20.252823189281116

Processing sentence: ['[CLS]', 'he', 'was', 'born', 'in', 'santiago', '.', '[SEP]']

Word: he 	 Prob_forward: -7.282431602478027; Prob_backwards: -0.27283912897109985
Word: was 	 Prob_forward: -0.49251610040664673; Prob_backwards: -7.999262743396685e-05
Word: 

-29.71513806514122

In [57]:
get_sentence_prob("I fed my cat some of it and he damn near passed out.")
get_sentence_prob("I fed my dog some of it and he damn near passed out.")
get_sentence_prob("I fed my window some of it and he damn near passed out.")
get_sentence_prob("I fed my the some of it and he damn near passed out.")

Processing sentence: ['[CLS]', 'i', 'fed', 'my', 'cat', 'some', 'of', 'it', 'and', 'he', 'damn', 'near', 'passed', 'out', '.', '[SEP]']

Word: i 	 Prob_forward: -7.554384708404541; Prob_backwards: -0.004160344135016203
Word: fed 	 Prob_forward: -10.962949752807617; Prob_backwards: -1.7643972635269165
Word: my 	 Prob_forward: -4.395064830780029; Prob_backwards: -2.2217588424682617
Word: cat 	 Prob_forward: -6.011991500854492; Prob_backwards: -13.184957504272461
Word: some 	 Prob_forward: -4.910660743713379; Prob_backwards: -5.519116401672363
Word: of 	 Prob_forward: -10.968462944030762; Prob_backwards: -2.6849160194396973
Word: it 	 Prob_forward: -8.965747833251953; Prob_backwards: -10.607772827148438
Word: and 	 Prob_forward: -0.6076505184173584; Prob_backwards: -0.6773967742919922
Word: he 	 Prob_forward: -5.089953422546387; Prob_backwards: -3.030817985534668
Word: damn 	 Prob_forward: -10.677398681640625; Prob_backwards: -0.3554331958293915
Word: near 	 Prob_forward: -0.8748708367347

-76.91740931871755

In [59]:
print("Should have similar/high probs\n")
get_sentence_prob("I forgot to take my medicine.")
get_sentence_prob("I forgot to take my medicines.")
get_sentence_prob("I forgot to take my medication.")
get_sentence_prob("I forgot to take my pills.")
get_sentence_prob("I forgot to take my turn.")
print("Should have low probs\n")
get_sentence_prob("I forgot to take my medical.")
get_sentence_prob("I forgot to take my medically.")
get_sentence_prob("I forgot to take my turned.")

Should have similar/high probs

Processing sentence: ['[CLS]', 'i', 'forgot', 'to', 'take', 'my', 'medicine', '.', '[SEP]']

Word: i 	 Prob_forward: -6.841585636138916; Prob_backwards: -0.009423289448022842
Word: forgot 	 Prob_forward: -8.716732025146484; Prob_backwards: -4.86270809173584
Word: to 	 Prob_forward: -2.7058005332946777; Prob_backwards: -0.22967574000358582
Word: take 	 Prob_forward: -2.5964887142181396; Prob_backwards: -4.419327735900879
Word: my 	 Prob_forward: -1.2803349494934082; Prob_backwards: -10.081429481506348
Word: medicine 	 Prob_forward: -4.346566677093506; Prob_backwards: -10.121175765991211
Word: . 	 Prob_forward: -0.002095509320497513; Prob_backwards: -0.3660288155078888
Raw forward sentence probability: -26.48960404470563
Raw backward sentence probability: -30.089768920093775

Average sentence log-prob: -28.289686482399702

Processing sentence: ['[CLS]', 'i', 'forgot', 'to', 'take', 'my', 'medicines', '.', '[SEP]']

Word: i 	 Prob_forward: -6.84158563613891

-36.512091025710106

In [54]:
get_sentence_prob("We will explore the elements used to construct sentences, and what parts of speech are used to expand and elaborate on them.")
get_sentence_prob("Wikipedia is a multilingual online encyclopedia created and maintained as an open collaboration project by a community of volunteer editors.")
get_sentence_prob("Once she gave her a little cap of red velvet, which suited her so well that she would never wear anything else.")

Processing sentence: ['[CLS]', 'we', 'will', 'explore', 'the', 'elements', 'used', 'to', 'construct', 'sentences', ',', 'and', 'what', 'parts', 'of', 'speech', 'are', 'used', 'to', 'expand', 'and', 'elaborate', 'on', 'them', '.', '[SEP]']

Word: we 	 Prob_forward: 0.0005438847583718598; Prob_backwards: 0.022272974252700806
Word: will 	 Prob_forward: 0.008487698622047901; Prob_backwards: 0.024045974016189575
Word: explore 	 Prob_forward: 9.5668867288623e-05; Prob_backwards: 0.0005457077058963478
Word: the 	 Prob_forward: 0.1310628205537796; Prob_backwards: 0.08164729177951813
Word: elements 	 Prob_forward: 1.3173785191611387e-05; Prob_backwards: 3.286916762590408e-05
Word: used 	 Prob_forward: 1.640969276195392e-05; Prob_backwards: 0.9119576811790466
Word: to 	 Prob_forward: 0.7283820509910583; Prob_backwards: 0.7407114505767822
Word: construct 	 Prob_forward: 0.004861429799348116; Prob_backwards: 0.0010620743269100785
Word: sentences 	 Prob_forward: 4.218660615151748e-05; Prob_backward

0.026338711941297832

In [15]:
# Load pre-trained model (weights)
with torch.no_grad():
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
#get_sentence_prob("I fed my cat some of it and he damn near passed out")
get_sentence_prob("He was born in Berlin.")
get_sentence_prob("He was born in Santiago.")
get_sentence_prob("He was born in France.")
get_sentence_prob("He was born in window.")
get_sentence_prob("He was born in was.")

Processing sentence: ['[CLS]', 'he', 'was', 'born', 'in', 'berlin', '.', '[SEP]']
Word: he 	 Prob_forward: 0.0021003505680710077; Prob_backwards: 0.8305719494819641

Word: was 	 Prob_forward: 0.21391922235488892; Prob_backwards: 0.9997050166130066

Word: born 	 Prob_forward: 0.00040079050813801587; Prob_backwards: 0.01874382235109806

Word: in 	 Prob_forward: 0.980291485786438; Prob_backwards: 0.018296195194125175

Word: berlin 	 Prob_forward: 0.02419455163180828; Prob_backwards: 9.928996587404981e-05

Word: . 	 Prob_forward: 0.9961835741996765; Prob_backwards: 0.9829498529434204


Geometric-mean forward sentence probability: 0.08986879181068508


Geometric-mean backward sentence probability: 0.11362871557646786


Average normalized sentence prob: 0.10105283461564618

Processing sentence: ['[CLS]', 'he', 'was', 'born', 'in', 'santiago', '.', '[SEP]']
Word: he 	 Prob_forward: 0.0021003505680710077; Prob_backwards: 0.7335055470466614

Word: was 	 Prob_forward: 0.21391922235488892; Prob_b

0.028633058233584703

In [17]:
get_sentence_prob("I fed my cat some of it and he damn near passed out.")
get_sentence_prob("I fed my dog some of it and he damn near passed out.")
get_sentence_prob("I fed my window some of it and he damn near passed out.")
get_sentence_prob("I fed my the some of it and he damn near passed out.")

Processing sentence: ['[CLS]', 'i', 'fed', 'my', 'cat', 'some', 'of', 'it', 'and', 'he', 'damn', 'near', 'passed', 'out', '.', '[SEP]']
Word: i 	 Prob_forward: 0.00788011122494936; Prob_backwards: 0.9571204781532288

Word: fed 	 Prob_forward: 0.0002093437797157094; Prob_backwards: 0.1745494306087494

Word: my 	 Prob_forward: 0.001939286245033145; Prob_backwards: 0.02095809206366539

Word: cat 	 Prob_forward: 0.0075833857990801334; Prob_backwards: 2.67108316620579e-06

Word: some 	 Prob_forward: 0.02979872189462185; Prob_backwards: 0.0016446709632873535

Word: of 	 Prob_forward: 0.002569902455434203; Prob_backwards: 0.014875434339046478

Word: it 	 Prob_forward: 0.0014934015925973654; Prob_backwards: 4.503015225054696e-05

Word: and 	 Prob_forward: 0.43204018473625183; Prob_backwards: 0.4202370047569275

Word: he 	 Prob_forward: 0.037493497133255005; Prob_backwards: 0.09001420438289642

Word: damn 	 Prob_forward: 3.998847751063295e-05; Prob_backwards: 0.8230639696121216

Word: near 	 Pr

0.013058838399427147

In [18]:
print("Should have similar/high probs\n")
get_sentence_prob("I forgot to take my medicine.")
get_sentence_prob("I forgot to take my medicines.")
get_sentence_prob("I forgot to take my medication.")
get_sentence_prob("I forgot to take my pills.")
print("Should have low probs\n")
get_sentence_prob("I forgot to take my turn.")
get_sentence_prob("I forgot to take my medical.")
get_sentence_prob("I forgot to take my medically.")
get_sentence_prob("I forgot to take my turned.")

Should have similar/high probs

Processing sentence: ['[CLS]', 'i', 'forgot', 'to', 'take', 'my', 'medicine', '.', '[SEP]']
Word: i 	 Prob_forward: 0.009378484450280666; Prob_backwards: 0.9732086658477783

Word: forgot 	 Prob_forward: 0.00021250064310152084; Prob_backwards: 0.015700489282608032

Word: to 	 Prob_forward: 0.03624541684985161; Prob_backwards: 0.055403366684913635

Word: take 	 Prob_forward: 0.017103174701333046; Prob_backwards: 0.02183195948600769

Word: my 	 Prob_forward: 0.00037714111385867; Prob_backwards: 2.8737551474478096e-05

Word: medicine 	 Prob_forward: 0.01692848652601242; Prob_backwards: 1.0145240594283678e-06

Word: . 	 Prob_forward: 0.9715754985809326; Prob_backwards: 0.9818249344825745


Geometric-mean forward sentence probability: 0.027014838021123058


Geometric-mean backward sentence probability: 0.02007291597720789


Average normalized sentence prob: 0.023286617911063946

Processing sentence: ['[CLS]', 'i', 'forgot', 'to', 'take', 'my', 'medicines', '.'

0.014468761225455259